In [ ]:
import fileinput
import os

tags_dict = {
          'TI': {'id' : 'TI', 'start' : '#ITI#', 'end' : '#FTI#'},
          'RE': {'id' : 'RE', 'start' : '#IRE#', 'end' : '#FRE#'},
          'PC': {'id' : 'PC', 'start' : '#IPC#', 'end' : '#FPC#'},
          'RF': {'id' : 'PC', 'start' : '#IRF#', 'end' : '#FRF#'}
}

def getElements(input_file):
  result = {}
  text_in_process = ''
  current_tag = ''

  for line in fileinput.input([input_file]):
    if line[2:4] in tags_dict.keys():
      current_tag = line[2:4]
    text_in_process += line
    if line[-6:-1] == tags_dict[current_tag]['end']:
      result[current_tag] = text_in_process.replace(tags_dict[current_tag]['start'], "").replace(tags_dict[current_tag]['end'], "")
      text_in_process = ''
      current_tag = ''

  return result

def generateOutputFiles(input_file, output_directory):
  if not os.path.exists(output_directory):
    os.makedirs(output_directory)
  doc_path = output_directory + '/' + input_file.rsplit('/', 1)[-1]
  #key_path = output_directory + '/' + input_file.rsplit('/', 1)[-1].replace('txt', 'key')
  content = getElements(input_file)
  with open(doc_path, 'w') as handler:
    handler.write('Title: ' + content['TI'] + '\n')
    handler.write('Abstract: ' + content['RE'] + '\n')

def generateOutputFilesFromDirectory(input_directory, output_directory):
  directory = os.fsencode(input_directory)
  for file in os.listdir(directory):
    filename = os.fsdecode(file)
    generateOutputFiles(input_directory + '/' + filename, output_directory)

In [ ]:
import bs4 as bs

def extractTextFromXml(xml_file_path, output_path):
    contents = {'title': '', 'abstract': '', 'body': ''}
    xml_data = None
    with open(xml_file_path, 'r', encoding="utf8") as xml_file:
        xml_data = xml_file.read()
    xml_soup = bs.BeautifulSoup(''.join(xml_data), "xml")
    el_title = xml_soup.find('article-title', {'xml:lang' : 'es'})
    if el_title == None:
        el_title = xml_soup.find('article-title')
    el_abstract = xml_soup.find('abstract', {'xml:lang' : 'es'})
    if el_abstract == None:
        el_abstract = xml_soup.find('abstract')
    if el_abstract != None:
        contents['abstract'] = el_abstract.text.strip()
    contents['title'] = el_title.text.strip()
    contents['body'] = xml_soup.find('body').text.strip()
    with open(output_path, 'w') as out_file:
        out_file.write(contents['title'] + '\n')
        out_file.write(contents['abstract'] + '\n')
        out_file.write(contents['body'])

In [ ]:
def generateOutputFilesFromXMLDirectory(input_directory:str, output_directory:str):
    file_id = ''
    try:
        for item in os.listdir(input_directory):
            out_file = output_directory
            if os.path.isdir(input_directory + item):
                input_files = input_directory + item + '/'
                if not os.path.exists(out_file + '/' + item):
                    os.makedirs(out_file + '/' + item)
                for file in os.listdir(input_files):
                    if file.endswith('.xml'):
                        file_id = file
                        out_file = out_file + '/' + item + '/' + file.replace('.xml', '').split(' ')[0] + '.txt'
                        extractTextFromXml(input_files + file, out_file)
            elif item.endswith('.xml'):
                    file_id = item
                    out_file = out_file + '/' + item.replace('.xml', '').split(' ')[0] + '.txt'
                    extractTextFromXml(input_directory + item, out_file)
    except Exception as ex:
        print(file_id, ex)